## RBM Name Generation

In [3]:
import argparse
import pickle
from sklearn.model_selection import train_test_split
from GridEncoder import GridEncoder
import Utils
from ShortTextCodec import ShortTextCodec, BinomialShortTextCodec
from RBM import BernoulliRBM
import Sampling
import sample
import sys
import colorama
import numpy as np
import sys
import os
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent =  currentdir + '\RBM_Git'
sys.path.insert(0,parent)
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import time
import matplotlib.pyplot as plt
import numexpr  as ne
import profile
import rbm as Rbm
import pandas
from random import randint
from timeit import default_timer as timer

In [ ]:
BIASED_PRIOR = 0

class CharBernoulli(BernoulliRBM):

    def __init__(self, **kwargs):
        """
        codec is the ShortTextCodec used to create the vectors being fit. The
        most important function of the codec is as a proxy to the shape of the
        softmax units in the visible layer (if you're using the CharBernoulliRBMSoftmax
        subclass). It's also used to decode and print
        fantasy particles at the end of each epoch.
        """
        # Attaching this to the object is really helpful later on when models
        # are loaded from pickle in visualize.py and sample.py
        self.codec = kwargs.pop("codec")
        self.softmax_shape = codec.shape()
        # Old-style class :(
        BernoulliRBM.__init__(self, **kwargs)

    def wellness_check(self, epoch, duration, train, validation):
        BernoulliRBM.wellness_check(self, epoch, duration, train, validation)
        fantasy_samples = '|'.join([self.codec.decode(vec) for vec in
                                    self._sample_visibles(self.h_samples_[:3], temperature=0.1)])
        print ("Fantasy samples: {}".format(fantasy_samples))

    def corrupt(self, v):
        n_softmax, n_opts = self.softmax_shape
        # Select a random index in to the indices of the non-zero values of each input
        # TODO: In the char-RBM case, if I wanted to really challenge the model, I would avoid selecting any
        # trailing spaces here. Cause any dumb model can figure out that it should assign high energy to
        # any instance of /  [^ ]/
        meta_indices_to_corrupt = self.rng_.randint(0, n_softmax, v.shape[0]) + np.arange(0, n_softmax * v.shape[0], n_softmax)

        # Offset these indices by a random amount (but not 0 - we want to actually change them)
        offsets = self.rng_.randint(1, n_opts, v.shape[0])
        # Also, do some math to make sure we don't "spill over" into a different softmax.
        # E.g. if n_opts=5, and we're corrupting index 3, we should choose offsets from {-3, -2, -1, +1}
        # 1-d array that matches with meta_i_t_c but which contains the indices themselves
        indices_to_corrupt = v.indices[meta_indices_to_corrupt]
        # Sweet lucifer
        offsets = offsets - (n_opts * (((indices_to_corrupt % n_opts) + offsets.ravel()) >= n_opts))

        v.indices[meta_indices_to_corrupt] += offsets
        return v, (meta_indices_to_corrupt, offsets)

    def uncorrupt(self, visibles, state):
        mitc, offsets = state
        visibles.indices[mitc] -= offsets
        
class CharBernoulliSoftmax(CharBernoulli):
    def __init__(self,**kwargs):
        CharBernoulli.__init__(self, **kwargs)
        
    def _sample_visibles(self, h, temperature=1.0):
        """Sample from the distribution P(v|h). This obeys the softmax constraint
        on visible units. i.e. sum(v) == softmax_shape[0] for any visible
        configuration v.

        h : array-like, shape (n_samples, n_components)
            Values of the hidden layer to sample from.

        Returns
        -------
        v : array-like, shape (n_samples, n_features)
            Values of the visible layer.
        """
        p = np.dot(h, self.components_/temperature)
        p += self.intercept_visible_/(min(1.0, temperature) if BIASED_PRIOR else temperature)
        nsamples, nfeats = p.shape
        reshaped = np.reshape(p, (nsamples,) + self.softmax_shape)
        return Utils.softmax_and_sample(reshaped).reshape((nsamples, nfeats))


In [ ]:
codec_kls = ShortTextCodec
codec = codec_kls('',10,0,True,False)
codec.debug_description()
model_kwargs = {'codec': codec,
                        'n_components': 300,
                        'learning_rate': 0.1,
                        'lr_backoff': False,
                        'n_iter': 500,
                        'verbose': 1,
                        'batch_size': 32,
                        'weight_cost': 0.0001,
                        }
kls = CharBernoulliSoftmax
rbm = kls(**model_kwargs)
codec.alphabet

### English Names File

In [ ]:
vecs = Utils.vectors_from_txtfile("./names.txt", codec)
train, validation = train_test_split(vecs, test_size=0.5)
print(train.shape,validation.shape)

In [ ]:
%%time
rbm.fit(train,validation)

In [ ]:
SAMPLES = []
def horizontal_cb(strings, i, energy=None):
    global SAMPLES
    
    if energy is not None:
        SAMPLES.append(zip(strings, energy))
    else:
        SAMPLES.append(strings)
        
def print_columns(maxlen):
    col_width = maxlen+2
    for fantasy_index in range(len(SAMPLES[0])):
        particles = [s[fantasy_index] for s in SAMPLES]
        print ("".join(s[fantasy_index].ljust(col_width) for s in SAMPLES))
        
sample_indices = [1000-1]
kwargs = dict(start_temp=1.0, final_temp=1.0, sample_energy=False, 
                    callback=horizontal_cb)

vis = Sampling.sample_model(rbm, 30, 1000, sample_indices, **kwargs)
print_columns(rbm.codec.maxlen)
fe = rbm._free_energy(vis)
print('Final energy: {:.2f} (stdev={:.2f})\n'.format(fe.mean(), fe.std()))

## Spanish Name File

In [ ]:
codec_kls = ShortTextCodec
codec = codec_kls('áéíóúñÁÉÍÓÚÑ',10,0,True,False)
codec.debug_description()
model_kwargs = {'codec': codec,
                        'n_components': 300,
                        'learning_rate': 0.1,
                        'lr_backoff': False,
                        'n_iter': 500,
                        'verbose': 1,
                        'batch_size': 32,
                        'weight_cost': 0.0001,
                        }
print(codec.alphabet)
kls = CharBernoulliSoftmax
rbm = kls(**model_kwargs)
vecs = Utils.vectors_from_txtfile("./spanish_Dict.txt", codec)
train, validation = train_test_split(vecs, test_size=0.5)
print(train.shape,validation.shape)

In [ ]:
%%time
rbm.fit(train,validation)

In [ ]:
SAMPLES = []
def horizontal_cb(strings, i, energy=None):
    global SAMPLES
    if energy is not None:
        SAMPLES.append(zip(strings, energy))
    else:
        SAMPLES.append(strings)
def print_columns(maxlen):
    col_width = maxlen+2
    for fantasy_index in range(len(SAMPLES[0])):
        particles = [s[fantasy_index] for s in SAMPLES]
        print ("".join(s[fantasy_index].ljust(col_width) for s in SAMPLES))
sample_indices = [1000-1]
kwargs = dict(start_temp=1.0, final_temp=1.0, sample_energy=False, 
                    callback=horizontal_cb)

vis = Sampling.sample_model(rbm, 30, 1000, sample_indices, **kwargs)
print_columns(rbm.codec.maxlen)
fe = rbm._free_energy(vis)
print('Final energy: {:.2f} (stdev={:.2f})\n'.format(fe.mean(), fe.std()))

## Test with RBM Implementation

In [4]:
import sys
import os
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent =  currentdir + '\RBM_Git'
sys.path.insert(0,parent)
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import time
import matplotlib.pyplot as plt
import numexpr  as ne
import profile
import rbm as Rbm
import pandas
from random import randint
from timeit import default_timer as timer

In [38]:
codec_kls = ShortTextCodec
codec = codec_kls('áéíóúñÁÉÍÓÚÑ',10,0,True,False)
print(codec.debug_description)
vecs = Utils.vectors_from_txtfile("./spanish_Dict.txt", codec)
visible_dim = vecs.shape[1]
hidden_dim = 200
epochs = 100
K = 1
lr = 0.1
batch_size = 10

<bound method ShortTextCodec.debug_description of <ShortTextCodec.ShortTextCodec object at 0x000002532C52BC88>>


In [6]:
rbm_ = Rbm.RBM(visible_dim=visible_dim,
               hidden_dim=hidden_dim,
               seed=42,
               mu=0, 
               sigma=0.3,
               monitor_time=True)
rbm_.W.shape, rbm_.b.shape, rbm_

((650, 200), (650,), <rbm.RBM at 0x2532c5ba160>)

In [7]:
test_Data_Vector_Aux = np.array(vecs.toarray(), dtype="float64")

In [8]:
%%time
rbm_.fit(test_Data_Vector_Aux, 
         method='CDK',
         K=K,
         lr=0.01,
         epochs=1,
         batch_size=128,
         plot_weights=False)

	Last epoch:ime per epoch: 107.44	total time: 107.45 0 	time per epoch: 107.44	total time: 107.45
	Training finished


Wall time: 1min 47s


In [9]:
%%time
rbm_.fit(test_Data_Vector_Aux, 
         method='vectorized_CDK',
         K=K,
         lr=0.01,
         epochs=1,
         batch_size=128,
         plot_weights=False)

	Last epoch:ime per epoch: 4.05	total time: 4.06 0 	time per epoch: 4.05	total time: 4.06
	Training finished


Wall time: 4.08 s


In [10]:
%%time
rbm_.fit(test_Data_Vector_Aux, 
         method='vectorized_CDK',
         K=K,
         lr=0.01,
         epochs=200,
         batch_size=128,
         plot_weights=False)

	epoch: 0 	time per epoch: 4.04	total time: 4.05 1 	time per epoch: 4.29	total time: 8.35 2 	time per epoch: 4.27	total time: 12.63 3 	time per epoch: 3.88	total time: 16.52 4 	time per epoch: 3.69	total time: 20.21 5 	time per epoch: 3.88	total time: 24.09 6 	time per epoch: 3.80	total time: 27.91 7 	time per epoch: 4.23	total time: 32.14 8 	time per epoch: 3.64	total time: 35.79 9 	time per epoch: 3.79	total time: 39.58 10 	time per epoch: 4.44	total time: 44.04 11 	time per epoch: 3.41	total time: 47.45 12 	time per epoch: 3.41	total time: 50.87 13 	time per epoch: 3.80	total time: 54.68 14 	time per epoch: 3.56	total time: 58.24 15 	time per epoch: 3.67	total time: 61.91 16 	time per epoch: 4.04	total time: 65.96 17 	time per epoch: 4.28	total time: 70.25 18 	time per epoch: 4.43	total time: 74.68 19 	time per epoch: 4.28	total time: 78.97 20 	time per epoch: 4.48	total time: 83.46 21 	time per epoch: 3.73	total time: 87.20 22 	time per epoch: 3.87	total time: 91.08 23 	time per ep

	Last epoch:	time per epoch: 4.91	total time: 782.99 186 	time per epoch: 5.39	total time: 788.38 187 	time per epoch: 5.28	total time: 793.67 188 	time per epoch: 4.56	total time: 798.23 189 	time per epoch: 4.41	total time: 802.64 190 	time per epoch: 4.09	total time: 806.74 191 	time per epoch: 4.58	total time: 811.33 192 	time per epoch: 4.67	total time: 816.00 193 	time per epoch: 4.48	total time: 820.48 194 	time per epoch: 5.15	total time: 825.63 195 	time per epoch: 4.48	total time: 830.12 196 	time per epoch: 4.39	total time: 834.51 197 	time per epoch: 4.38	total time: 838.90 198 	time per epoch: 4.21	total time: 843.11 199 	time per epoch: 4.44	total time: 847.56 199 	time per epoch: 4.44	total time: 847.56
	Training finished


Wall time: 14min 7s


In [22]:
word = codec.encode_onehot("acabar")
print(word.shape)

(650,)


In [44]:
x_hat, x_hat_p = rbm_.sample_visible_from_visible(word, n_gibbs=2000)
final_result = []
for i in range(x_hat.shape[0]):
    if x_hat[i] == True:
        final_result.append(1)
    else:
        final_result.append(0)
result = []
result.append(final_result)
result = np.asarray(result)
result = result.ravel()
print(result)
print(codec.decode(result))

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [40]:
fW = codec.decode(x_hat)
print(fW)
# for i in range(10):
#     x_hat1,x_hat_p1 = rbm_.sample_visible_from_visible(x_hat, n_gibbs=2000)
#     print(codec.decode(x_hat1))

?os???$$$$


## SKLEARN BERNOULLI

In [46]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import BernoulliRBM
%matplotlib inline   
import pandas as pd

In [47]:
rbm_Bernoulli = BernoulliRBM(n_components=100, learning_rate=0.01, random_state=0, verbose=True)

In [48]:
vecs = Utils.vectors_from_txtfile("./spanish_Dict.txt", codec)

In [49]:
rbm_Bernoulli.fit(vecs)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -40.73, time = 9.06s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -39.50, time = 9.74s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -39.88, time = 9.68s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -38.82, time = 9.71s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -37.48, time = 11.55s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -37.71, time = 10.01s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -37.60, time = 12.08s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -36.75, time = 11.66s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -36.95, time = 10.31s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -35.77, time = 10.59s


BernoulliRBM(batch_size=10, learning_rate=0.01, n_components=100, n_iter=10,
       random_state=0, verbose=True)

In [60]:
test = rbm_Bernoulli.gibbs(vecs[0])
test = test.ravel()

z?l?$$$$$$


In [72]:
def transformFromOneHotVector(vec):
    finalWord = []
    for k in range(len(vec)):
        tmp = []
        for i in range(len(vec[k])):
            if vec[k][i] == 1:
                index = i%maxLength
                if(index != maxLength):
                    tmp.append(index)
        finalWord.append(tmp)
    return finalWord

In [77]:
xx = vecs[:40].copy()
maxLength = 26
for ii in range(1000):
    for n in range(40):
        xx[n] = rbm_Bernoulli.gibbs(xx[n])
for i in range(xx.shape[0]):
    tmp = xx[i].toarray()
    print(transformFromOneHotVector(tmp))
print(xx.shape)

C:\Users\mferrer\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


[[25, 7, 19, 4, 13, 0, 13, 0, 13]]
[[25, 13, 7, 13, 0, 13, 0, 13]]
[[25, 7, 13, 17, 2, 25, 13, 13, 0, 13, 0, 13]]
[[21, 25, 1, 7, 12, 0, 13, 0, 13, 0, 13]]
[[25, 7, 17, 4, 7, 13, 0, 13, 0, 13]]
[[25, 7, 14, 18, 4, 13, 0, 13, 0, 13]]
[[25, 13, 13, 14, 25, 13, 0, 13, 0, 13]]
[[25, 7, 17, 1, 14, 13, 0, 13, 0, 13]]
[[25, 0, 13, 0, 13, 0, 13]]
[[25, 1, 7, 1, 0, 13, 0, 13, 0, 13]]
[[25, 7, 2, 1, 7, 25, 13, 0, 13, 0, 13]]
[[25, 17, 19, 13, 0, 0, 13, 0, 13]]
[[25, 7, 13, 0, 13, 13, 0, 13, 0, 13]]
[[25, 14, 1, 7, 17, 17, 13, 0, 13, 0, 13]]
[[20, 25, 2, 13, 0, 13, 0, 13]]
[[6, 13, 0, 13, 0, 13, 0, 13]]
[[25, 7, 1, 13, 0, 13, 0, 13]]
[[25, 13, 7, 11, 13, 18, 4, 13, 0, 13, 0, 13]]
[[25, 13, 0, 13, 0, 13]]
[[21, 25, 13, 7, 3, 17, 6, 21, 0, 10, 13, 0, 13, 0, 13]]
[[25, 7, 2, 4, 13, 0, 13, 0, 13]]
[[25, 7, 18, 12, 13, 14, 0, 13, 0, 13, 0, 13]]
[[25, 1, 17, 13, 0, 13, 0, 13, 0, 13]]
[[25, 17, 19, 0, 25, 13, 0, 13, 0, 13]]
[[7, 12, 1, 2, 0, 1, 13, 0, 13, 0, 13]]
[[25, 7, 2, 3, 17, 1, 17, 0, 13, 0, 13, 

## BERNOULLI GRIDS TEST

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import BernoulliRBM
import numpy as np

In [5]:
def generate_data_fake(): 
    vec = [] 
    for i in range(1000): 
        tmp = [] 
        for k in range(48): 
            tmp.append(0) 
        vec.append(tmp) 
    return np.asarray(vec) 

grid_data = generate_data_fake()
for i in range(1000):
    grid_data[i][0] = 1

In [6]:
oneHotEncoder = OneHotEncoder(255, sparse=True).fit(grid_data)
oneHotData = oneHotEncoder.transform(grid_data)

In [8]:
RBM_Machine = BernoulliRBM(n_components=250, learning_rate=0.01, n_iter=10, random_state=0, verbose=True)

In [9]:
RBM_Machine.fit(oneHotData)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -408.82, time = 9.01s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -85.96, time = 8.68s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -30.28, time = 8.16s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -18.74, time = 8.09s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -13.28, time = 9.24s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -9.87, time = 8.46s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -8.17, time = 7.87s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -7.04, time = 10.00s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -6.09, time = 9.27s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -5.42, time = 8.66s


BernoulliRBM(batch_size=10, learning_rate=0.01, n_components=250, n_iter=10,
       random_state=0, verbose=True)

In [10]:
x_visible = RBM_Machine.gibbs(oneHotData[0])
x_visible = x_visible.ravel()

In [11]:
final_vector = []
for k in range(len(x_visible)):
    if(x_visible[k] == True):
        final_vector.append(1)
    else:
        final_vector.append(0)
output_data = []
for i in range(48):
    tmp = final_vector[(255*i):((i+1)*255)]
    output_data.append(tmp.index(1))
print (output_data)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
